## Lesson: 
- **Step-by-step guide to generating an interactive climate map in Bokeh (& Geopandas)**
- **Code already filled in** (to make sure that it works), but students will be given a version of this notebook with less code pre-written.

### STEP 1: Read in historical temperature data
- This data was already scraped from Berkeley Earth and then post-processed to make this activity a bit easier.
- Make sure students understand what it is (pretty self-explanatory)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../data/all_country_temp_data_CLEAN.csv')

In [ ]:
df.head()

### STEP 2: Read in the geographic data (geometric shapes of all countries in the world) 
- Shape file (.shp)
    - https://en.wikipedia.org/wiki/Shapefile#Shapefile_shape_format_(.shp)
- We will need a specialized library in Python for handling geographic data -- **GeoPandas**
- Installing GeoPandas: http://geopandas.org/install.html

In [ ]:
import geopandas as gpd
SHAPEFILE = '../data/ne_110m_admin_0_countries.shp'
gdf = gpd.read_file(SHAPEFILE)[['ADMIN', 'geometry']]
#we only need country name and the geometry

In [ ]:
gdf.columns = ['country', 'geometry']

In [ ]:
gdf.head()

### STEP 3: Group / aggregate the temperature anomaly data by year
- For simplicity, we're only interested in yearly averages

In [ ]:
df = df.groupby(['country', 'year'])[['monthly_anomaly']].mean().reset_index()

### STEP 4: Merge Data Sets.
- We want to have our temperature data and geometric data in one place.
- Make sure you're still left with a GeoDataFrame at the end.

In [ ]:
gdf = pd.merge(left=gdf, right=df, how='left', on='country')

In [ ]:
gdf.head()

---
---
### Time for Visualization

---
---

### STEP 5: Let's see if we can plot data on a map for a single year.
- We'll make it interactive later
- The Bokeh library (as well as many other JavaScript-based mapping libraries) require the data to be in GeoJSON format.
- So let's generate a GeoJSON for a single year.
- **Three Steps:**
    1. Generate a blank figure
    2. Add country shapes / polygons onto the map
    3. Color the shapes according to temperature values

#### 5a. Generate a blank canvas / figure.

In [ ]:
from bokeh.plotting import figure

#Create a blank figure object.
p = figure(title = 'Avg. Monthly Temperature Anomaly for Year 2000',
           plot_height = 600,
           plot_width = 1000,
          )

In [ ]:
#Display figure inline in Jupyter Notebook.
from bokeh.io import output_notebook, show

output_notebook()
show(p)

#### 5b. Generate a GeoJSON for a single year and use it to add geometry layer onto the figure

In [ ]:
json_2000 = gdf[gdf['year'] == 2000].to_json()

In [ ]:
from bokeh.models import GeoJSONDataSource
geosource = GeoJSONDataSource(geojson = json_2000)

In [ ]:
#Add patch renderer to figure. Like actually add the map to the canvas.
p.patches('xs',
          'ys',
          source = geosource,
          line_color = 'black',
          line_width = 0.25)

In [ ]:
show(p)

### 5c. Let's associate values with some colors

In [ ]:
from bokeh.palettes import brewer

In [ ]:
palette = brewer['RdBu'][9]

In [ ]:
from bokeh.models import LinearColorMapper
color_mapper = LinearColorMapper(palette = palette,
                                 low = -3,
                                 high = 3, 
                                 nan_color = '#d9d9d9')

In [ ]:
from bokeh.models import ColorBar
color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=8,
                     width = 500,
                     height = 20,
                     border_line_color=None,
                     location = (0,0),
                     orientation = 'horizontal'
                    )

In [ ]:
p.add_layout(color_bar, 'below')
show(p)

In [ ]:
p.patches('xs',
          'ys',
          source = geosource,
          fill_color = {'field' :'monthly_anomaly', 'transform':color_mapper}, ### NEW ###
          line_color = 'black',
          line_width = 0.25)

In [ ]:
show(p)

### STEP 6: Add Interactivity so that we can change attributes of the map with a slider
- Bokeh provides an extensive set of widgets and tools and makes it very simple to create rich, interactive visualizations. We will define a few functions and reuse a major chunk of code written for creating the static map.
- Three Steps:
    1. Let's make our lives easier by defining a function that changes the data (i.e. GeoJSON) based on year.
    2. Add a slider widget that we can interact with
    3. Write a function that controls what changes when we move the slider.

#### 6a. Function

In [ ]:
def get_geojson(yr):
    """Input a year (int) and return corresponding slice of the GeoDataFrame, converted to GeoJSON"""
    gdf_year = gdf[gdf['year'] == yr]
    json_data = gdf_year.to_json()
    return json_data

geosource = GeoJSONDataSource(geojson = get_geojson(2000))

In [ ]:
### OLD CODE AGAIN FOR REFERENCE:

p = figure(title = 'Avg. Monthly Temperature Anomaly for Year 2000',
           plot_height = 600,
           plot_width = 1000)


palette = brewer['RdBu'][9]

color_mapper = LinearColorMapper(palette = palette,
                                 low = -3,
                                 high = 3, 
                                 nan_color = '#d9d9d9')

color_bar = ColorBar(color_mapper=color_mapper,
                     label_standoff=8,
                     width = 500,
                     height = 20,
                     border_line_color=None,
                     location = (0,0),
                     orientation = 'horizontal'
                    )

p.patches('xs',
          'ys',
          source = geosource,
          fill_color = {'field' :'monthly_anomaly', 'transform':color_mapper}, ### NEW ###
          line_color = 'black',
          line_width = 0.25)

p.add_layout(color_bar, 'below')

#### 6b. Slider Widget

In [ ]:
from bokeh.models import Slider
slider = Slider(title = 'Year', start = 1900, end = 2013, step = 1, value = 1900)

#### 6c. "Callback" Function
- Now, the really cool features about the slider is that you can put in your own, custom callback function!!
- So basically, all we have to do is write our own python function that does whatever we want to happen everytime the slider is adjusted.

In [ ]:
def update_plot(attr, old, new):
    """Change properties / attributes of the datasource and title depending on slider value / position."""
    yr = slider.value
    new_data = get_geojson(yr)
    geosource.geojson = new_data
    p.title.text = f'Avg. Monthly Temperature Anomaly for Year {yr}'
      
slider.on_change('value', update_plot)

### And finally, just some boilerplate code to wrap everything together...

In [ ]:
from bokeh.layouts import widgetbox, column
from bokeh.io import curdoc

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p,widgetbox(slider))
curdoc().add_root(layout)

In [ ]:
show(layout)

In the terminal, run:

``bokeh serve --show interactive_climate_map_LESSON.ipynb``

### Bonus / Follow-up:
- Add a hover tool (so data is shown when the mouse hovers over a country)
- Any other cool widgets you can think of?
- Get more data up through 2019/2020.
    - Any data source / API where you might be able to get this?
- Create predictions through 2050, and add them to the visualization.
- Why does the data load slowly, and how could we improve the speed?